In [ ]:
import numpy as np
import galois
from functools import reduce
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, pairing, final_exponentiate, Z1, neg, FQ12

# 1. Configuration & Helper Functions

# The scalar field order of the BN128 curve. 
# All arithmetic regarding the "exponents" (scalars) must happen in this field.
ORDER = curve_order 

# Initialize the Galois Field with the curve order
GF = galois.GF(ORDER)

def inner_product(points, coeffs):
    """
    Computes the Multi-Scalar Multiplication (MSM).
    Sum(coeff[i] * point[i])
    """
    # converting galois ints to standard python ints for py_ecc
    return reduce(add, (multiply(point, int(coeff)) for point, coeff in zip(points, coeffs)), Z1)

In [2]:
# Problem: Prove we know x, y such that x^2 + y^2 = z
# Witness vector structure: [1, z, x, y, v1, v2]
# Witness Values:           [1, 25, 3, 4, 9, 16]

L = np.array([
    [0, 0, 1, 0, 0, 0], # x
    [0, 0, 0, 1, 0, 0], # y
    [0, 0, 0, 0, 1, 1]  # v1 + v2
])

R = np.array([
    [0, 0, 1, 0, 0, 0], # x
    [0, 0, 0, 1, 0, 0], # y
    [1, 0, 0, 0, 0, 0]  # 1
])

O = np.array([
    [0, 0, 0, 0, 1, 0], # v1
    [0, 0, 0, 0, 0, 1], # v2
    [0, 1, 0, 0, 0, 0]  # z
])

witness_vec = np.array([1, 25, 3, 4, 9, 16])

# Convert matrices to Galois Field for arithmetic
L_field = GF(L)
R_field = GF(R)
O_field = GF(O)
w_field = GF(witness_vec)


In [3]:
def interpolate_columns(matrix):
    """
    Converts columns of the matrix into polynomials using Lagrange interpolation.
    For a matrix with 3 rows (constraints), we interpolate over domain x = {1, 2, 3}.
    """
    num_rows, num_cols = matrix.shape
    xs = GF(np.arange(1, num_rows + 1))
    
    polys = []
    for i in range(num_cols):
        ys = matrix[:, i]
        poly = galois.lagrange_poly(xs, ys)
        polys.append(poly)
    return polys

U_polys = interpolate_columns(L_field)
V_polys = interpolate_columns(R_field)
W_polys = interpolate_columns(O_field)

# Calculate the Target Polynomial T(x)
# T(x) = (x-1)(x-2)(x-3) for 3 constraints
T = galois.Poly([1], field=GF)
for i in range(1, L.shape[0] + 1):
    T *= galois.Poly([1, -i], field=GF)



In [4]:
def get_accumulated_poly(polys, witness):
    """Computes Sum(w_i * P_i(x))"""
    res = galois.Poly([0], field=GF)
    for poly, w in zip(polys, witness):
        res += poly * w
    return res

Pu = get_accumulated_poly(U_polys, w_field)
Pv = get_accumulated_poly(V_polys, w_field)
Pw = get_accumulated_poly(W_polys, w_field)

# Division to find H
# The remainder must be zero for the witness to be valid
H = (Pu * Pv - Pw) // T
remainder = (Pu * Pv - Pw) % T

assert remainder == 0, "R1CS constraints not satisfied!"


In [5]:
# 1. Toxic Waste (In reality, these are deleted immediately)
tau = GF(12345) 
alpha = GF(500)
beta = GF(600)
gamma = GF(700)
delta = GF(800)

def evaluate_poly_at_tau(polys, tau_val):
    return [poly(tau_val) for poly in polys]

# Pre-evaluate polynomials at tau
u_at_tau = evaluate_poly_at_tau(U_polys, tau)
v_at_tau = evaluate_poly_at_tau(V_polys, tau)
w_at_tau = evaluate_poly_at_tau(W_polys, tau)

# 2. SRS Generation

# Powers of tau for H(x) coverage: mapped to G1
# specific calculation: (tau^i * T(tau)) / delta
degree_T = T.degree
t_at_tau = T(tau)
srs_H = []
for i in range(degree_T): 
    val = (tau**i * t_at_tau) / delta
    srs_H.append(multiply(G1, int(val)))

# Proving Key (Private inputs: indices 2..end)
# Calculation: (beta*u_i(tau) + alpha*v_i(tau) + w_i(tau)) / delta
srs_private_L = []
for i in range(2, len(witness_vec)):
    val = (beta * u_at_tau[i] + alpha * v_at_tau[i] + w_at_tau[i]) / delta
    srs_private_L.append(multiply(G1, int(val)))

# Verification Key (Public inputs: indices 0..1)
# Calculation: (beta*u_i(tau) + alpha*v_i(tau) + w_i(tau)) / gamma
srs_public_ic = []
for i in range(2): 
    val = (beta * u_at_tau[i] + alpha * v_at_tau[i] + w_at_tau[i]) / gamma
    srs_public_ic.append(multiply(G1, int(val)))

# Verification Key Constants
vk_alpha_g1 = multiply(G1, int(alpha))
vk_beta_g2 = multiply(G2, int(beta))
vk_gamma_g2 = multiply(G2, int(gamma))
vk_delta_g2 = multiply(G2, int(delta))


In [ ]:
# Prover chooses random blinding factors
r = GF(99)
s = GF(77)

# Split witness into public and private
w_pub = w_field[:2]
w_priv = w_field[2:]

# Compute A_1
u_sum = sum([w_field[i] * u_at_tau[i] for i in range(len(w_field))], GF(0))
A_val = alpha + u_sum + r * delta
proof_A = multiply(G1, int(A_val))

# Compute B_2
v_sum = sum([w_field[i] * v_at_tau[i] for i in range(len(w_field))], GF(0))
B_val = beta + v_sum + s * delta
proof_B = multiply(G2, int(B_val))
 
# Compute B_1
proof_B1 = multiply(G1, int(B_val))

# 4. Compute C_1
# C = (Sum_priv(w_i * L_i) + H(tau)*T(tau)/delta) + A*s + B*r - r*s*delta

# The Private Witness Linear Combination
term_1 = inner_product(srs_private_L, w_priv)

# The H(x) 
term_2 = inner_product(srs_H, H.coeffs[::-1]) 

# Blinding terms
term_3 = multiply(proof_A, int(s))
term_4 = multiply(proof_B1, int(r))

# We calculate the negative scalar in the field, then convert to int
rs_neg = -(r * s) 
term_5 = multiply(multiply(G1, int(delta)), int(rs_neg))

# Sum
proof_C = add(term_1, term_2)
proof_C = add(proof_C, term_3)
proof_C = add(proof_C, term_4)
proof_C = add(proof_C, term_5)


In [10]:
# Compute Linear Combination of Public Inputs
# L_pub = Sum_public(w_i * IC_i)
L_pub = inner_product(srs_public_ic, w_pub)

# Pairing Check
# e(A, B) * e(alpha, -beta) * e(L_pub, -gamma) * e(C, -delta) == 1

term_AB = pairing(proof_B, proof_A)
term_alpha_beta = pairing(vk_beta_g2, neg(vk_alpha_g1))
term_pub = pairing(vk_gamma_g2, neg(L_pub))
term_delta = pairing(vk_delta_g2, neg(proof_C))

final_result = final_exponentiate(term_AB * term_alpha_beta * term_pub * term_delta)

is_valid = final_result == FQ12.one()

print(f"Proof Verification Result: {is_valid}")
if is_valid:
    print("SUCCESS: The verifier is convinced you know x^2 + y^2 = 25")
else:
    print("FAILURE: Proof rejected.")

Proof Verification Result: True
SUCCESS: The verifier is convinced you know x^2 + y^2 = 25
